In [24]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import svm
from sklearn import linear_model, dummy, metrics
from sklearn.dummy import DummyClassifier
from sklearn.metrics import *
import time
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [25]:
df = pd.read_csv(r"training.csv", decimal = ',')

In [26]:
df.variable3 = df.variable3.replace(',','.', regex=True)
df['variable3'] = df['variable3'].apply(pd.to_numeric,errors='coerce')

del df['variable18']
df = df.drop(columns=['variable3', 'variable8'])
df = df.drop(columns=['variable10','variable17'])

df = df.drop(columns=['variable19'],axis=1)
df = df.dropna()
df['classLabel'].replace({'no.': 0, 'yes.': 1},inplace = True)

#assuming that t means 'TRUE', and f indicates 'FALSE' in variable9, variable10, and variable12. t = 1, f = 0
df['variable9'].replace({'f': 0, 't': 1},inplace = True)
df['variable12'].replace({'f': 0, 't': 1},inplace = True)

#cleaning rest of non-numeric variables
categorical_columns = ['variable1','variable4', 'variable5', 'variable6', 'variable7', 'variable13']
# transform the categorical columns
df = pd.get_dummies(df, columns=categorical_columns)

#Using MinMaxScaler to standardize our data
scaler = preprocessing.MinMaxScaler()
scaler.fit(df)
df = pd.DataFrame(scaler.transform(df), index=df.index, columns=df.columns)

X = df.loc[:,df.columns!='classLabel']
y = df[['classLabel']]

In [28]:
#Decision Tree Classifier

t0=time.time()
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)
clf_dt = DecisionTreeClassifier(random_state=0) #logisticRegr
clf_dt.fit(X_train,y_train)
y_pred = clf_dt.predict(X_test) 

In [30]:
#Input File
dfv = pd.read_csv(r"validation.csv", decimal = ',')

In [31]:
##Cleaning input File

dfv.variable3 = dfv.variable3.replace(',','.', regex=True)
dfv['variable3'] = dfv['variable3'].apply(pd.to_numeric,errors='coerce')
dfv = dfv.drop(columns=['variable3', 'variable8','variable10','variable17','variable18','variable19']) 
dfv = dfv.dropna()
dfv['classLabel'].replace({'no.': 0, 'yes.': 1},inplace = True)
dfv['variable9'].replace({'f': 0, 't': 1},inplace = True)
dfv['variable12'].replace({'f': 0, 't': 1},inplace = True)


categorical_columns = ['variable1','variable4', 'variable5', 'variable6', 'variable7', 'variable13']
dfv = pd.get_dummies(dfv, columns=categorical_columns)


scaler = preprocessing.MinMaxScaler()
scaler.fit(dfv)
dfv = pd.DataFrame(scaler.transform(dfv), index=dfv.index, columns=dfv.columns)

#Converting this to trained mode format
df2 = df.iloc[0:0]
df3 = pd.concat([df2,dfv], sort = False)
df3 = df3.fillna(0)

In [34]:
#Input X and y
X_val = df3.loc[:,df3.columns!='classLabel']
y_val = df3[['classLabel']]
y_predicted = clf_dt.predict(X_val)

y_predicted_df = pd.DataFrame({'Predicted':y_predicted})
y_val.reset_index(drop=True, inplace=True)
result = pd.concat([y_val, y_predicted_df], axis=1)
result.to_csv(r"result.csv")